In [ ]:

# Écrit requirements.txt (selon l'énoncé) puis installe.
from pathlib import Path
requirements = """
langgraph
langchain
langchain_core
langchain_community
langchainhub
ipykernel
langchain_groq
langchain_huggingface
beautifulsoup4
tiktoken
chromadb
langchain_google_genai
python-dotenv
ipywidgets
"""
Path("requirements.txt").write_text(requirements)

import sys, subprocess
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])
    print("Dépendances installées.")
except Exception as e:
    print("Installation ignorée ou partiellement réussie:", e)



# Configuration `.env`
> Le provider est auto-détecté dans l’ordre: `groq` si clé présente, sinon `google`, sinon `hf`.


In [ ]:

# (Optionnel) Génère un template .env si absent.
from pathlib import Path
if not Path(".env").exists():
    Path(".env").write_text(
        "LLM_PROVIDER=\n"
        "GROQ_API_KEY=\n"
        "GOOGLE_API_KEY=\n"
        "HUGGINGFACEHUB_API_TOKEN=\n"
    )
    print("Template .env créé. Renseignez vos clés avant d'exécuter la suite.")
else:
    print(".env déjà présent.")



# Imports
Imports principaux (conformes à l’énoncé) et utilitaires.


In [ ]:

# Imports demandés
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# Imports utilitaires et LLM/Embeddings
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableConfig
from langchain_core.documents import Document
from langchain_core.messages import AIMessage
from langchain_core.tools import tool

# Aide debug/affichage
from pprint import pprint



# Initialisation LLM et Embeddings
Sélection dynamique du provider (Groq / Google / HuggingFace). Les embeddings utilisent un modèle `sentence-transformers` standard.


In [ ]:

load_dotenv()

def get_chat_model():
    """Retourne un ChatModel LangChain en fonction des clés présentes et/ou LLM_PROVIDER."""
    provider = os.getenv("LLM_PROVIDER", "").strip().lower()
    groq_key = os.getenv("GROQ_API_KEY", "").strip()
    google_key = os.getenv("GOOGLE_API_KEY", "").strip()
    hf_key = os.getenv("HUGGINGFACEHUB_API_TOKEN", "").strip()

    # Auto-détection si provider non spécifié
    if not provider:
        if groq_key:
            provider = "groq"
        elif google_key:
            provider = "google"
        else:
            provider = "hf"

    if provider == "groq":
        # Modèles recommandés: llama-3.1-70b-versatile, llama-3.1-8b-instant, mixtral-8x7b-32768
        return ChatGroq(model="llama-3.1-8b-instant", temperature=0)
    elif provider == "google":
        # Modèles possibles: gemini-1.5-pro, gemini-1.5-flash
        return ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
    else:
        # Fallback simple via Hugging Face (non outillé pour tool-calling avancé, mais suffisant ici)
        # Vous pouvez substituer par un provider local si nécessaire.
        from langchain_huggingface import HuggingFaceEndpoint
        return HuggingFaceEndpoint(
            repo_id="mistralai/Mistral-7B-Instruct-v0.2",
            temperature=0,
            max_new_tokens=512,
        )

def get_embeddings():
    """Embeddings standard pour Chroma."""
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

chat_model = get_chat_model()
embeddings = get_embeddings()

print(f"LLM prêt: {chat_model.__class__.__name__}")
print("Embeddings prêts.")



# Construction de la base de connaissances (R de RAG)
1. Chargement web avec `WebBaseLoader`.
2. Découpage (`RecursiveCharacterTextSplitter`).
3. Vectorisation avec Chroma.
4. Création de l’outil de recherche `retriever_tool`.


In [ ]:

# 1) URLs sources (exemples). Adaptez selon vos besoins.
URLS = [
    # Articles techniques ou de blog (exemples publics) :
    "https://python.langchain.com/docs/get_started/introduction/",
    "https://python.langchain.com/docs/integrations/vectorstores/chroma/",
]

# 2) Chargement
loader = WebBaseLoader(URLS)
docs = loader.load()

# 3) Découpage
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
chunks = splitter.split_documents(docs)

# 4) Vectorstore Chroma (persist_dir pour réutiliser)
PERSIST_DIR = "chroma_db_autocorrect_rag"
vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=PERSIST_DIR)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 5) Outil retriever
retriever_tool = create_retriever_tool(
    retriever,
    name="knowledge_search",
    description=(
        "Recherche des passages pertinents dans la base de connaissances interne. "
        "Utilisez cet outil pour répondre aux questions nécessitant des informations issues des documents ingérés."
    ),
)

print(f"Documents chargés: {len(docs)} | Chunks: {len(chunks)}")



# Définition de l’état (mémoire) de l’agent
`AgentState` contient l’historique des messages (géré par `add_messages`) et un compteur d’itérations anti-boucle infinie.


In [ ]:

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    iteration: int

# Helpers pour extraire le dernier ToolMessage (résultats du retriever).
def get_last_tool_payload(messages: Sequence[BaseMessage]) -> str:
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            # Le contenu du ToolMessage est souvent une chaîne (formatée par le tool)
            return msg.content if isinstance(msg.content, str) else str(msg.content)
    return ""



# Nœuds et logique
# 1) `assistant` (router)
Modèle outillé: décide de répondre directement ou d’appeler l’outil (`knowledge_search`).

# 2) `retrieve`
`ToolNode` préconstruit qui exécute l’outil de recherche.

# 3) `grade_documents` (aiguillage conditionnel)
LLM à sortie structurée pour décider si les résultats sont pertinents.

# 4) `rewrite`
Réécrit la question si la récupération est jugée non pertinente.

# 5) `generate`
Synthétise la réponse finale à partir des documents validés.


In [ ]:

# 1) Assistant (router) : LLM avec outil bindé
assistant_llm = chat_model.bind_tools([retriever_tool])

def ai_assistant(state: AgentState) -> AgentState:
    """Nœud principal: appelle le LLM outillé. Si besoin, le LLM déclenche l'outil de recherche."""
    messages = state["messages"]
    response = assistant_llm.invoke(messages)
    return {"messages": [response], "iteration": state.get("iteration", 0)}

# 2) Retrieve: ToolNode prêt à l'emploi
tool_node = ToolNode([retriever_tool])

# 3) Grading structuré
class GradeDecision(BaseModel):
    relevant: Literal["yes", "no"] = Field(..., description="yes si les documents sont pertinents, no sinon.")
    reasoning: str = Field(..., description="Explication courte sur la pertinence.")

grade_prompt = PromptTemplate.from_template(
    """Vous êtes un évaluateur de recherche. On vous donne:

- Question utilisateur:
{question}

- Résultats de la recherche (texte brut):
{tool_output}

Tâche:
1) Indiquez si les résultats sont pertinents pour répondre à la question.
2) Répondez strictement via un schéma structuré.

Exigences:
- relevant: "yes" si globalement pertinent, sinon "no".
- reasoning: explication concise.
"""
)

grade_chain = grade_prompt | chat_model.with_structured_output(GradeDecision)

def grade_documents(state: AgentState) -> str:
    """Aiguilleur conditionnel: renvoie 'generate' si pertinent, sinon 'rewrite'."""
    last_tool = get_last_tool_payload(state["messages")
    question = ""
    # Retrouve la dernière question utilisateur
    for msg in reversed(state["messages"]):
        if isinstance(msg, HumanMessage):
            question = msg.content
            break
    if not last_tool:
        # Si pas de payload outil, mieux vaut réécrire pour clarifier la requête.
        return "rewrite"

    decision = grade_chain.invoke({"question": question, "tool_output": last_tool})
    route = "generate" if decision.relevant == "yes" else "rewrite"
    # On log l'analyse dans les messages (optionnel, pour audit)
    analysis_note = AIMessage(content=f"[Grading] relevant={decision.relevant} | {decision.reasoning}")
    return route

# 4) Rewrite node
rewrite_prompt = PromptTemplate.from_template(
    """Réécrivez la question suivante pour l'améliorer (plus claire, précise, contextuelle) en vue d'une recherche documentaire.

Question:
{question}

Renvoyez UNIQUEMENT la nouvelle question, sans commentaire.
"""
)

def rewrite_question(state: AgentState) -> AgentState:
    # Dernière question utilisateur
    question = ""
    for msg in reversed(state["messages"]):
        if isinstance(msg, HumanMessage):
            question = msg.content
            break
    improved = (rewrite_prompt | chat_model | StrOutputParser()).invoke({"question": question})
    # Ajoute la version améliorée en tant que nouveau message utilisateur pour relancer la boucle
    return {
        "messages": [HumanMessage(content=improved)],
        "iteration": state.get("iteration", 0) + 1,
    }

# 5) Generate node
answer_prompt = PromptTemplate.from_template(
    """Vous êtes un assistant utile. Répondez de manière complète et structurée à la question en vous appuyant sur le CONTEXTE fourni.

CONTEXTE (résultats de recherche bruts):
{context}

QUESTION:
{question}

CONSIGNES:
- Citez vos sources si disponibles (liens/URL dans le contexte).
- Si l'information manque, dites-le explicitement et proposez une piste de reformulation.
"""
)

def generate(state: AgentState) -> AgentState:
    # Récupère la dernière question + le dernier résultat d'outil
    question = ""
    for msg in reversed(state["messages"]):
        if isinstance(msg, HumanMessage):
            question = msg.content
            break
    context = get_last_tool_payload(state["messages"]) or "(Aucun contexte disponible)"
    answer = (answer_prompt | chat_model | StrOutputParser()).invoke({"context": context, "question": question})
    return {"messages": [AIMessage(content=answer)], "iteration": state.get("iteration", 0)}



# Construction du graphe LangGraph
- `START → assistant`  
- `assistant` via `tools_condition` → `tools` si un outil est appelé, sinon `generate`  
- `tools` → aiguillage conditionnel via `grade_documents`: `generate` si pertinent, `rewrite` sinon  
- `rewrite` → `assistant` (boucle d’auto-correction)  
- `generate` → `END`


In [ ]:

# Assemble le StateGraph
graph = StateGraph(AgentState)

graph.add_node("assistant", ai_assistant)
graph.add_node("tools", tool_node)
graph.add_node("rewrite", rewrite_question)
graph.add_node("generate", generate)

graph.add_edge(START, "assistant")

# tools_condition envoie "tools" si le LLM a demandé un tool-call, sinon __else__
graph.add_conditional_edges(
    "assistant",
    tools_condition,
    {"tools": "tools", "__else__": "generate"},
)

# Après exécution du tool, on grade
graph.add_conditional_edges(
    "tools",
    grade_documents,
    {"generate": "generate", "rewrite": "rewrite"},
)

# Boucle de correction
graph.add_edge("rewrite", "assistant")

# Clôture
graph.add_edge("generate", END)

# Compile l'application
app = graph.compile()

print("Graphe compilé.")



# Exécution et Tests
La fonction utilitaire `run_query` facilite les tests. Vous pouvez aussi activer le stream pour observer le chemin emprunté.


In [ ]:

def run_query(query: str, stream: bool = True, recursion_limit: int = 15):
    """Exécute le graphe sur une requête utilisateur et affiche le résultat final.


    - stream=True pour tracer les transitions.

    - recursion_limit pour éviter les boucles infinies.

    """
    init_state: AgentState = {"messages": [HumanMessage(content=query)], "iteration": 0}
    cfg = {"recursion_limit": recursion_limit}

    if stream:
        print("\n--- STREAM START ---")
        events = app.stream(init_state, config=cfg, stream_mode="values")
        final = None
        for step in events:
            # step est un dict partiel de l'état
            msgs = step.get("messages", [])
            if msgs:
                last = msgs[-1]
                print(f"[{last.__class__.__name__}] {getattr(last, 'name', '')}\n{last.content[:300]}...\n")
            final = step
        print("--- STREAM END ---\n")
        return final
    else:
        result = app.invoke(init_state, config=cfg)
        return result

# Exemples de tests rapides (adaptez selon les contenus ingérés)
tests = [
    "Bonjour !",  # salutation simple (devrait répondre sans outil)
    "Comment utiliser Chroma avec LangChain ?",  # précis, covered by docs
    "Parle-moi de vectorisation pour la recherche documentaire",  # vague: peut déclencher rewrite
    "Quels sont les entraînements de l'équipe nationale de rugby ?",  # hors-sujet
]

for i, q in enumerate(tests, 1):
    print(f"===== TEST {i}: {q} =====")
    _ = run_query(q, stream=True)



# Observations
- Si la question est claire et couverte par la base, l’agent répond directement ou appelle l’outil puis génère la réponse.
- Si la récupération est non pertinente, la réécriture s’active et boucle jusqu’à obtenir des documents mieux ciblés (ou jusqu’à la limite de récursion).
- Le grading utilise une sortie structurée (`relevant: yes/no`) pour piloter la logique interne de l’agent.
- Vous pouvez enrichir `URLS` avec vos propres sources, ou brancher des loaders (PDF, dossiers…) pour un RAG plus étoffé.



# Bonus : Intégration Streamlit (optionnel)
Exécutez la cellule ci-dessous pour générer un fichier `app.py`.  
Lancez ensuite localement : `streamlit run app.py`


In [ ]:

app_code = """
import os
import streamlit as st
from langchain_core.messages import HumanMessage
from main import app  # Si vous placez le graphe dans un module 'main.py'. Ici, on va recharger localement si besoin.

st.set_page_config(page_title="Agent RAG Auto-Correcteur", page_icon=None, layout="centered")

st.title("Agent RAG Auto-Correcteur (LangGraph)")
st.write("Posez une question. L'agent cherchera, évaluera, réécrira si nécessaire, puis répondra.")

query = st.text_area("Votre question", height=100, placeholder="Ex: Comment utiliser Chroma avec LangChain ?")
go = st.button("Envoyer" )

if go and query.strip():
    with st.spinner("Traitement..."):
        from langchain_core.messages import HumanMessage
        init_state = {"messages": [HumanMessage(content=query)], "iteration": 0}
        result = app.invoke(init_state, config={"recursion_limit": 15})
        msgs = result.get("messages", [])
        if msgs:
            st.subheader("Réponse")
            st.write(msgs[-1].content)
        else:
            st.warning("Aucune réponse générée.")
"""

# Écrit un app.py minimal (nécessite que ce notebook soit converti en module 'main.py' ou
# adaptez pour importer 'app' depuis la cellule courante).
from pathlib import Path
Path("app.py").write_text(app_code)
print("Fichier app.py écrit. Adaptez l'import du graphe si vous l'extrayez dans un module.")
